## If a model, after cross-validated, performs badly in the test data. Can we go back and tune the model parameters/hyper-parameters further so that it performs better in the test data?

(probably not...)

## What is a good CV strategy for time-series?

(Look at MLEDU Slides 1, Page 41, among others)